In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [4]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [5]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
115305,Tariq Lamptey,1,2,0,Brighton and Hove Albion,Burnley,0.549968,0.283075,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115306,Taylor Richards,1,3,0,Brighton and Hove Albion,Burnley,0.549968,0.283075,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115307,Teemu Pukki,1,4,76,Norwich,Liverpool,0.328739,1.998371,True,2,...,0.0,0.0,0.0,10.0,11.0,90.9,12.0,0.0,1.0,1
115308,Theo Walcott,1,3,79,Southampton,Everton,0.532456,1.098383,False,2,...,0.0,0.0,0.0,12.0,20.0,60.0,12.0,0.0,0.0,1
115309,Thiago Alcántara do Nascimento,1,3,0,Liverpool,Norwich,1.998371,0.328739,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115310,Thiago Emiliano da Silva,1,2,0,Chelsea,Crystal Palace,2.083714,0.477592,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115311,Thomas Partey,1,3,0,Arsenal,Brentford,1.190377,0.372963,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115312,Timothy Castagne,1,2,0,Leicester City,Wolverhampton Wanderers,1.230500,0.889458,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115313,Timo Werner,1,4,90,Chelsea,Crystal Palace,2.083714,0.477592,True,2,...,0.0,1.0,0.0,13.0,21.0,61.9,17.0,1.0,1.0,1
115314,Tim Krul,1,1,90,Norwich,Liverpool,0.328739,1.998371,True,2,...,0.0,0.0,0.0,30.0,44.0,68.2,24.0,0.0,0.0,1


In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

2

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
df_latest.head(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Aaron Connolly,2,4,45,Brighton and Hove Albion,Watford,0.538249,0.322706,True,1,...,15599,0,2,6.0,-1682,1899,3581,0,2021-08-21T16:30:00Z,2122
1,Aaron Cresswell,2,2,90,West Ham United,Leicester City,0.628252,1.233477,True,2,...,1345295,1,4,6.0,21799,76490,54691,0,2021-08-23T19:00:00Z,2122
2,Aaron Ramsdale,2,1,0,Arsenal,Chelsea,1.268682,2.025905,True,0,...,1432,2,0,0.0,597,679,82,0,2021-08-22T15:30:00Z,2122
3,Aaron Ramsey,2,3,0,Aston Villa,Newcastle United,0.950515,0.534485,True,0,...,11767,0,2,0.0,-384,1171,1555,0,2021-08-21T14:00:00Z,2122
4,Aaron Wan-Bissaka,2,2,90,Manchester United,Southampton,2.115023,0.543120,False,2,...,707386,1,1,2.0,-47625,17513,65138,0,2021-08-22T13:00:00Z,2122
5,Abdoulaye Doucouré,2,3,90,Everton,Leeds,1.095428,0.538581,False,4,...,71179,2,2,16.0,25889,32845,6956,1,2021-08-21T14:00:00Z,2122
6,Adama Traoré,2,3,90,Wolverhampton Wanderers,Tottenham Hotspur,0.858519,1.558729,True,2,...,85215,1,0,65.0,-2356,9369,11725,0,2021-08-22T13:00:00Z,2122
7,Adam Armstrong,2,4,90,Southampton,Manchester United,0.543120,2.115023,True,2,...,165181,1,1,26.0,56591,77504,20913,0,2021-08-22T13:00:00Z,2122
8,Adam Forshaw,2,3,0,Leeds,Everton,0.538581,1.095428,True,0,...,503,2,2,0.0,141,184,43,0,2021-08-21T14:00:00Z,2122
9,Adam Idah,2,4,0,Norwich,Manchester City,0.341636,2.346950,False,0,...,36741,0,5,0.0,-623,5296,5919,0,2021-08-21T14:00:00Z,2122


In [10]:
df_latest.shape

(566, 34)

In [11]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [12]:
last_play_proba.head()

,player,team,play_proba
0,James Milner,Liverpool,1.0
1,Jordan Henderson,Liverpool,1.0
2,Adrián San Miguel del Castillo,Liverpool,1.0
3,Joel Matip,Liverpool,1.0
4,Thiago Alcántara do Nascimento,Liverpool,1.0


In [13]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [14]:
df_latest.drop_duplicates(inplace=True)

In [15]:
df_train.drop_duplicates(inplace=True)

In [16]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [17]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [18]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [19]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
West Ham
https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats
Vladimír Coufal
Aaron Cresswell
Craig Dawson
Łukasz Fabiański
Pablo Fornals
Angelo Ogbonna
Declan Rice
Tomáš Souček
Jarrod Bowen
Michail Antonio
Saïd Benrahma
Ryan Fredericks
Mark Noble
Andriy Yarmolenko
Ben Johnson
Chelsea
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Andreas Christensen
Jorginho
Edouard Mendy
Antonio Rüdiger
Marcos Alonso
Mason Mount
Mateo Kovačić
César Azpilicueta
Reece James
Kai Havertz
Timo Werner
Trevoh Chalobah
Romelu Lukaku
Christian Pulisic
N'Golo Kanté
Hakim Ziyech
Emerson Palmieri
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Trent Alexander-Arnold
Alisson
Virgil van Dijk
Sadio Mané
Joël Matip
Mohamed Salah
Kostas Tsimikas
Naby Keïta
Diogo Jota
Harvey Elliot
Jordan Henderson
James Milner
Alex Oxlade-Chamberlain
Roberto Firmino
Fabinho
Thiago Alcántara
Joe Gomez
Brighton
https://fbref.com/en/squads/d07537b9/Brig

In [20]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [21]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [22]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [23]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [24]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Vladimír Coufal,2021-08-15,West Ham United,Premier League,1,0,0,67,11,...,0.6,1,0,42,61,68.9,42,1,1,Vladimir Coufal
1,2021-2022,Vladimír Coufal,2021-08-23,West Ham United,Premier League,2,0,0,89,11,...,0.6,5,1,63,83,75.9,51,0,0,Vladimir Coufal
2,2021-2022,Aaron Cresswell,2021-08-15,West Ham United,Premier League,1,2,1,90,9,...,0.1,2,0,67,79,84.8,54,0,1,Aaron Cresswell
3,2021-2022,Aaron Cresswell,2021-08-23,West Ham United,Premier League,2,1,0,81,3,...,0.0,1,1,65,76,85.5,50,0,0,Aaron Cresswell
4,2021-2022,Craig Dawson,2021-08-15,West Ham United,Premier League,1,1,0,59,8,...,0.0,1,0,45,48,93.8,37,0,0,Craig Dawson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,2021-2022,Bali Mumba,2021-08-21,Norwich,Premier League,2,0,0,19,12,...,0.0,0,0,7,10,70.0,9,0,1,Bali Mumba
551,2021-2022,Josh Sargent,2021-08-14,Norwich,Premier League,1,1,0,11,6,...,0.0,0,0,4,4,100.0,4,0,1,Joshua Sargent
552,2021-2022,Josh Sargent,2021-08-21,Norwich,Premier League,2,0,0,7,2,...,0.0,0,0,4,5,80.0,5,1,1,Joshua Sargent
553,2021-2022,Adam Idah,2021-08-14,Norwich,Premier League,1,1,1,6,5,...,0.0,0,0,3,5,60.0,2,0,0,Adam Idah


In [25]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [30]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

[]

In [27]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [28]:
# any additional corrections
# match_list['Imran Louza'] = 'Imrân Louza'
# match_list['Oghenekaro Etebo'] = 'Oghenekaro Peter Etebo'
# match_list['Ismaila Sarr'] = 'Ismaila Sarr'
# match_list['Cucho'] = 'Juan Camilo Hernández Suárez'
# match_list['Enoch Mwepu'] = 'Enock Mwepu'
# match_list['Emi Buendía'] = 'Emiliano Buendía Stati'
# match_list['José Sá'] = 'José Malheiro de Sá'
# match_list['Francisco Trincão'] = 'Francisco Machado Mota de Castro Trincão'
# match_list['Valentino Livramento'] = 'Tino Livramento'
# match_list['Nuno Tavares'] = 'Nuno Varela Tavares'
# match_list['Jesuran Rak Sakyi'] = 'Jesurun Rak-Sakyi'
# match_list['Josh Sargent'] = 'Joshua Sargent'
# match_list['Junior Firpo'] = 'Héctor Junior Firpo Adames'

In [29]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [67]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

[]

In [31]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [32]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [33]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [34]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [35]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [36]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [37]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [38]:
# add crowds field
df_latest_further['crowds'] = 1

In [39]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [40]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [41]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
113008,Harry Kane,36,4,90,Tottenham Hotspur,Wolverhampton Wanderers,1.586460,0.960312,True,8,...,0.1,6,1,18,27,66.7,28,3,5,0
113629,Harry Kane,37,4,90,Tottenham Hotspur,Aston Villa,1.586543,0.884857,True,2,...,0.2,1,0,11,24,45.8,21,1,1,1
114301,Harry Kane,38,4,90,Tottenham Hotspur,Leicester City,1.568259,1.165630,False,10,...,0.1,3,2,8,16,50,28,6,7,1
114999,Harry Kane,1,4,0,Tottenham Hotspur,Manchester City,1.560569,2.349720,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115556,Harry Kane,2,4,18,Tottenham Hotspur,Wolverhampton Wanderers,1.558729,0.858519,False,0,...,0.1,2,0,7,10,70.0,7,0,0,1


In [42]:
df_train_new.tail(7)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
115914,Yerson Mosquera Valdelamar,2,2,0,Wolverhampton Wanderers,Tottenham Hotspur,0.858519,1.558729,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115915,Yoane Wissa,2,3,13,Brentford,Crystal Palace,0.372523,0.477029,False,1,...,0.0,0,0,2,3,66.7,4,1,1,1
115916,Youri Tielemans,2,3,90,Leicester City,West Ham United,1.233477,0.628252,False,7,...,0.0,2,1,38,48,79.2,48,0,0,1
115917,Yves Bissouma,2,3,90,Brighton and Hove Albion,Watford,0.538249,0.322706,True,8,...,0.4,3,2,47,55,85.5,44,2,2,1
115918,Zack Steffen,2,1,0,Manchester City,Norwich,2.346950,0.341636,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115919,Álvaro Fernández,2,1,0,Brentford,Crystal Palace,0.372523,0.477029,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115920,Çaglar Söyüncü,2,2,90,Leicester City,West Ham United,1.233477,0.628252,False,0,...,0.0,0,0,85,99,85.9,80,0,0,1


In [43]:
df_latest_further.shape

(566, 57)

In [44]:
df_train_new.shape

(115921, 59)

In [45]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [46]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [47]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
20,Manchester City,Arsenal,2021-08-28T11:30:00Z,3,0
21,Aston Villa,Brentford,2021-08-28T14:00:00Z,3,1
22,Brighton and Hove Albion,Everton,2021-08-28T14:00:00Z,3,2
23,Newcastle United,Southampton,2021-08-28T14:00:00Z,3,3
24,Norwich,Leicester City,2021-08-28T14:00:00Z,3,4
25,West Ham United,Crystal Palace,2021-08-28T14:00:00Z,3,5
26,Liverpool,Chelsea,2021-08-28T16:30:00Z,3,6
27,Burnley,Leeds,2021-08-29T13:00:00Z,3,7
28,Tottenham Hotspur,Watford,2021-08-29T13:00:00Z,3,8
29,Wolverhampton Wanderers,Manchester United,2021-08-29T15:30:00Z,3,9


In [48]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [49]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,43,3
1,3,1,Aston Villa,Brentford,2021-08-28T14:00:00Z,7,94
2,3,2,Brighton and Hove Albion,Everton,2021-08-28T14:00:00Z,36,11
3,3,3,Newcastle United,Southampton,2021-08-28T14:00:00Z,4,20
4,3,4,Norwich,Leicester City,2021-08-28T14:00:00Z,45,13


In [50]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [51]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
14,Thomas Partey,0,3,15,50,3,Partey
17,Lucas Torreira,0,3,18,45,3,Torreira
18,Edward Nketiah,0,4,20,55,3,Nketiah
22,Konstantinos Mavropanos,0,2,24,45,3,Mavropanos
23,Mattéo Guendouzi,0,3,25,45,3,Guendouzi
...,...,...,...,...,...,...,...
539,Rui Pedro dos Santos Patrício,0,1,427,50,39,Patrício
545,Jonathan Castro Otto,0,2,433,45,39,Jonny
553,Pedro Lomba Neto,0,3,441,65,39,Neto
554,Owen Otasowie,0,3,442,45,39,Otasowie


In [52]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [53]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [54]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,3,0,Arsenal,Manchester City,2021-08-28T11:30:00Z,Bernd Leno,1,49,None,Leno,False
1,3,0,Arsenal,Manchester City,2021-08-28T11:30:00Z,Rúnar Alex Rúnarsson,1,40,100,Rúnarsson,False
2,3,0,Arsenal,Manchester City,2021-08-28T11:30:00Z,Willian Borges Da Silva,3,63,50,Willian,False
3,3,0,Arsenal,Manchester City,2021-08-28T11:30:00Z,Pierre-Emerick Aubameyang,4,99,100,Aubameyang,False
4,3,0,Arsenal,Manchester City,2021-08-28T11:30:00Z,Cédric Soares,2,45,None,Cédric,False


In [55]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Jack Grealish,3,80,None,Grealish,True
1,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Kyle Walker,2,55,None,Walker,True
2,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Ilkay Gündogan,3,74,100,Gündogan,True
3,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Kevin De Bruyne,3,120,0,De Bruyne,True
4,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,John Stones,2,54,None,Stones,True


In [56]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [57]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [58]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Jack Grealish,3,8.0,1.00,Grealish,True,2.346950,1.268682,2122
1,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Kyle Walker,2,5.5,1.00,Walker,True,2.346950,1.268682,2122
2,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Ilkay Gündogan,3,7.4,1.00,Gündogan,True,2.346950,1.268682,2122
3,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Kevin De Bruyne,3,12.0,0.00,De Bruyne,True,2.346950,1.268682,2122
4,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,John Stones,2,5.4,1.00,Stones,True,2.346950,1.268682,2122
5,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Benjamin Mendy,2,5.5,1.00,Mendy,True,2.346950,1.268682,2122
6,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Riyad Mahrez,3,9.0,1.00,Mahrez,True,2.346950,1.268682,2122
7,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Raheem Sterling,3,11.0,1.00,Sterling,True,2.346950,1.268682,2122
8,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,João Pedro Cavaco Cancelo,2,6.0,1.00,Cancelo,True,2.346950,1.268682,2122
9,3,0,Manchester City,Arsenal,2021-08-28T11:30:00Z,Ederson Santana de Moraes,1,6.0,1.00,Ederson,True,2.346950,1.268682,2122


In [59]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')